In [1]:
%cd ..
%load_ext autoreload
%autoreload 2
# %load_ext nb_black
# notebook setup

from pathlib import Path
import sys

sys.path.insert(0, Path().absolute().parent.as_posix())

/Users/ruizhechao/Documents/NNforHJB


/Users/ruizhechao/Documents/NNforHJB/.venv/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
# load the data
import numpy as np

path = 'rawdata/raw_data/data/VDP_beta_0.1_grid_30x30.npy'
data = np.load(path)
len(data)

900

In [3]:
# prepare the data
# data is a structured numpy array with fields: 'x', 'dv', 'v'
# convert to the dict format expected by model._prepare_data

data_dict = {
    "x": np.asarray(data["x"], dtype=np.float64),    # shape (N, 2)
    "v": np.asarray(data["v"], dtype=np.float64),    # shape (N,)
    "dv": np.asarray(data["dv"], dtype=np.float64),  # shape (N, 2)
}

## Test effect of different gamma with increased neurons added

In [4]:
gammas = [10.0, 1.0, 1e-1, 1e-2, 0]
alpha = 1e-3
power = 2.1
loss_weight_h1 = [1.0, 1.0]
loss_weight_l2 = [1.0, 0.0]

num_iterations = 10
num_insertions = 50
pruning_threshold = 1e-10

In [5]:
# pdpa_list_l2 = []
# best_iteration_l2 = []
# for gamma in gammas:
#     pdpa = PDPA(
#         data=data_dict,
#         alpha=alpha,
#         gamma=gamma,
#         power=power,
#         activation=torch.relu,
#         loss_weights=loss_weight_l2,
#         verbose=False,
#     )

#     best_iteration = pdpa.retrain(
#     num_iterations = num_iterations, 
#     num_insertion= num_insertions, 
#     threshold = pruning_threshold,
#     verbose=False
#     )

#     pdpa_list_l2.append(pdpa)
#     best_iteration_l2.append(best_iteration)


In [6]:
# best_iteration_l2

In [7]:
import torch
from scr.PDPA import PDPA

pdpa_list_h1 = []
best_iteration_h1 = []
best_neurons_h1 = []

for gamma in gammas:
    pdpa = PDPA(
        data=data_dict,
        alpha=alpha,
        gamma=gamma,
        power=power,
        activation=torch.relu,
        loss_weights=loss_weight_h1,
        verbose=False,
    )

    best_iteration, best_neurons = pdpa.retrain(
        num_iterations = num_iterations, 
        num_insertion= num_insertions, 
        threshold = pruning_threshold,
        verbose=False
    )

    pdpa_list_h1.append(pdpa)
    best_iteration_h1.append(best_iteration)
    best_neurons_h1.append(best_neurons)

In [8]:
best_iteration_h1

[3, 1, 1, 1, 0]

In [9]:
best_neurons_h1

[183, 99, 98, 98, 49]

In [10]:
import pickle

# model_l2 = {
#     "gammas": gammas,
#     "pdpa_list_l2": pdpa_list_l2,
#     "alpha": alpha,
#     "power": power,
#     "num_iteration": num_iterations,
#     "num_insertion": num_insertions,
#     "pruning_threshold": pruning_threshold,
#     "best_iteration_l2": best_iteration_l2,
# }

model_h1 = {
    "gammas": gammas,
    "pdpa_list_h1": pdpa_list_h1,
    "alpha": alpha,
    "power": power,
    "num_iteration": num_iterations,
    "num_insertion": num_insertions,
    "pruning_threshold": pruning_threshold,
    "best_iteration_h1": best_iteration_h1,
}

# with open("models/pdpa_vdp_model_l2_7.pkl", "wb") as f:
#     pickle.dump(model_l2, f)

with open("models/experiment_5/pdpa_vdp_model_h1_1.pkl", "wb") as f:
    pickle.dump(model_h1, f)
